# Association Analysis

The object of this notebook is to begin an association analysis to see how the actors, directors and writers affected the user score, critic score, box office (US and global) and IMDB votes the movie had earned.

### Modules We'll Be Using

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
import sklearn
import math
import re
import time

# names_df = pd.read_csv('IMDb names.csv')
# ratings_df = pd.read_csv('IMDb ratings.csv')
# titlePrinciples_df = pd.read_csv('IMDb title_principals.csv')

### Pre-processing the Movies CSV

First Let's read in our data and convert the values we care about up front:

In [2]:
# First Read it in
movies_df = pd.read_csv('IMDb movies.csv')

# Rename Columns and convert data types where appropriate
movies_df = movies_df.astype(
    {
     'duration' : 'float',
     'avg_vote' : 'float',
     'votes' : 'float',
#      'budget' : 'float', # Need this to replace '$ \d+' with float(\d+)
#      'usa_gross_income' : 'float', # Need this to replace '$ \d+' with float(\d+)
#      'worlwide_gross_income' : 'float', # Need this to replace '$ \d+' with float(\d+)
     'metascore' : 'float',
     'reviews_from_users' : 'float',
     'reviews_from_critics' : 'float',
    }).rename(columns=
    {'imdb_title_id' : 'title_id',
     'reviews_from_users' : 'user_reviews',
     'reviews_from_critics' : 'critic_reviews'
    })

movies_df = movies_df.set_index(movies_df['title_id']).rename(columns={'title_id' : 'REMOVE'}).drop(columns=['REMOVE'])

# Found a bad row that didn't contain an int for a year (or a year formatted as such)
movies_df = movies_df.drop(index=['tt8206668'], axis=1)
movies_df = movies_df.astype({'year' : 'int'})

# We also don't want any older films to keep the data relatively current
movies_df = movies_df[movies_df['year'] > 1970]

# Make sure to remove all rows where our fields of interest are blank
movies_df = movies_df.dropna(subset=['actors', 'director', 'writer', 'usa_gross_income'])

# Now print a quick summary
print(f'Shape: {movies_df.shape} \n{movies_df.columns}')
movies_df[['title', 'director', 'writer', 'actors']].head()

C:\Users\jcjwg\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Shape: (14478, 21) 
Index(['title', 'original_title', 'year', 'date_published', 'genre',
       'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'user_reviews', 'critic_reviews'],
      dtype='object')


,title,director,writer,actors
title_id,,,,
tt0035423,Kate & Leopold,James Mangold,"Steven Rogers, James Mangold","Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki..."
tt0064321,Sesso in faccia,Al Adamson,"Raphael Nussbaum, Jale Lockwood","Lon Chaney Jr., Russ Tamblyn, Jennifer Bishop,..."
tt0064820,Tutti contro Harry,Michael Roemer,Michael Roemer,"Martin Priest, Ben Lang, Maxine Woods, Henry N..."
tt0065117,Nuda dal fiume,"Ján Kadár, Elmar Klos","Imre Gyöngyössy, Ján Kadár","Rade Markovic, Milena Dravic, Paula Pritchett,..."
tt0065750,La furia del Hombre Lobo,José María Zabalza,Paul Naschy,"Paul Naschy, Perla Cristal, Verónica Luján, Mi..."


Now we can create our market baskets from the movies summary we just built.  The transaaction ID in these baskets will still be the 'title_id', and we'll keep the respective scores, dollar values, dates, etc. intact.  We will create one for each seperate category of interest and one for all of them together.  Let's first start with the separate ones

In [3]:
actor_only_df = movies_df.drop(columns=['director', 'writer'])
writer_only_df = movies_df.drop(columns=['director', 'actors'])
director_only_df = movies_df.drop(columns=['writer', 'actors'])

# Only consider films with over 10000 votes
actor_only_df = actor_only_df[director_only_df['votes'] >= 10000]
writer_only_df = writer_only_df[director_only_df['votes'] >= 10000]
director_only_df = director_only_df[director_only_df['votes'] >= 10000]

# # Only consider films with a Meta Critic, user, critic and average review score
actor_only_df = actor_only_df.dropna(subset=['metascore', 'user_reviews', 'critic_reviews', 'avg_vote'])
writer_only_df = writer_only_df.dropna(subset=['metascore', 'user_reviews', 'critic_reviews', 'avg_vote'])
director_only_df = director_only_df.dropna(subset=['metascore', 'user_reviews', 'critic_reviews', 'avg_vote'])

# Also drop any blank entries from their respective tables
actor_only_df = actor_only_df.dropna(subset=['actors'])
writer_only_df = writer_only_df.dropna(subset=['writer'])
director_only_df = director_only_df.dropna(subset=['director'])

actor_only_df.head(3)

,title,original_title,year,date_published,genre,duration,country,language,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,user_reviews,critic_reviews
title_id,,,,,,,,,,,,,,,,,,,
tt0035423,Kate & Leopold,Kate & Leopold,2001,2002-03-01,"Comedy, Fantasy, Romance",118.0,USA,"English, French",Konrad Pictures,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",An English Duke from 1876 is inadvertedly drag...,6.4,77852.0,$ 48000000,$ 47121859,$ 76019048,44.0,341.0,115.0
tt0066434,L'uomo che fuggì dal futuro,THX 1138,1971,1976-10-29,"Drama, Sci-Fi, Thriller",86.0,USA,English,American Zoetrope,"Robert Duvall, Donald Pleasence, Don Pedro Col...","In the twenty-fifth century, a time when peopl...",6.7,47108.0,$ 777000,$ 2437000,$ 2437000,75.0,232.0,83.0
tt0066819,La notte brava del soldato Jonathan,The Beguiled,1971,1971-01-23,"Drama, Thriller, War",105.0,USA,"English, French",The Malpaso Company,"Clint Eastwood, Geraldine Page, Elizabeth Hart...",While imprisoned in a Confederate girls' board...,7.2,15421.0,NaN,$ 1100000,$ 1113069,66.0,95.0,69.0


### Helper Functions

In [4]:
# Helper function to create a market basket out of a given dataframe, column and delimiter
def make_marketbasket_from_list(df, colname, delimiter):
    starttime = time.time()
    basket_df = df[colname]
    itemlist = []
    
    # First get the list of items that'll constitute our columns
    for trans_id in basket_df.index:
        itemset = re.split(delimiter, basket_df[trans_id])

        for item in itemset:
            if (item.lower() not in itemlist):
                itemlist.append(item.lower())

    print(f'Item Count: {len(itemlist)}')

    # Now make the transactions by creating an np.array of boolean values
    itemtrans = []
    
    for trans_id in basket_df.index:
        trans = []
        
        if (math.floor(len(itemtrans) % (df.count().max() / 10)) == 0):
            print(f'{len(itemtrans)}/{df.count().max()}, Runtime: {time.time() - starttime}sec')
            starttime = time.time()

        for i in range(len(itemlist)):
            trans.append(bool(
                re.match(
                    re.compile(f'{itemlist[i]}'),
                    basket_df[trans_id].lower()
                )
            ))
        
        itemtrans.append(trans)
    
    itemtrans = np.array(itemtrans)
    print(f'Basket Shape: {itemtrans.shape}')
    
    # Finally, return the market basket!
    return(pd.DataFrame(index=df.index, columns=itemlist, data=itemtrans))

# Helper function to make interval market basket data
def make_marketbasket_from_interval(df, colname, lowend, highend, step):
    starttime = time.time()
    basket_df = df[colname]
    lowend = float(lowend)
    highend = float(highend)
    step = float(step)
    dividers = np.arange(lowend, highend + 1, step)
    
    # Make the baskets
    basket = [f'{colname} < {lowend}']
    
    for i in range(len(dividers) - 1):
        basket.append(f'{colname} in [{dividers[i]}, {dividers[i + 1]})')
    
    basket.append(f'{colname} >= {highend}')
    print(f'Discretized Basket: {basket}')

    # Now make the transactions
    itemtrans = []

    for tid in basket_df.index:
        if (math.floor(len(itemtrans) % (df.count().max() / 10)) == 0):
            print(f'{len(itemtrans)}/{df.count().max()}, Runtime: {time.time() - starttime}sec')
            starttime = time.time()

        trans = [bool(lowend > basket_df[tid])]

        for i in range(len(dividers) - 1):
            trans.append(
                bool((dividers[i] <= basket_df[tid]) and (dividers[i + 1] > basket_df[tid]))
            )
        
        trans.append(bool(highend <= basket_df[tid]))
        itemtrans.append(trans)

    itemtrans = np.array(itemtrans)
    print(f'Transaction Shape: {itemtrans.shape}')
    
    # Finally, return the market basket!
    return(pd.DataFrame(index=df.index, columns=basket, data=itemtrans))

# Helper function to substitute a column with its market basket
def convert_to_marketbasket_list(df, col_basket, delim):
    marketbasket = make_marketbasket_from_list(df, col_basket, delim)
    
    return(df.merge(marketbasket, how='outer', left_index=True, right_index=True).drop(columns=[col_basket]))

# Helper function that substitutes column with interval market basket
def convert_to_marketbasket_interval(df, col_basket, lowend, highend, step):
    marketbasket = make_marketbasket_from_interval(df, col_basket, lowend, highend, step)
    
    return(df.merge(marketbasket, how='outer', left_index=True, right_index=True).drop(columns=[col_basket]))

#### Let's First make the scoring Market Baskets

In [5]:
%%time
# The three persons' baskets
director_basket = make_marketbasket_from_list(director_only_df , 'director', ', ')
actor_basket = make_marketbasket_from_list(actor_only_df , 'actors', ', ')
writer_basket = make_marketbasket_from_list(writer_only_df , 'writer', ', ')

Item Count: 2467
0/5988, Runtime: 0.15659880638122559sec
599/5988, Runtime: 93.18459272384644sec
1198/5988, Runtime: 89.00901246070862sec
1797/5988, Runtime: 88.8885977268219sec
2396/5988, Runtime: 96.30799984931946sec
2994/5988, Runtime: 93.29938077926636sec
3593/5988, Runtime: 94.46380925178528sec
4192/5988, Runtime: 105.72726774215698sec
4791/5988, Runtime: 95.13960146903992sec
5390/5988, Runtime: 95.19225358963013sec
Basket Shape: (5988, 2467)
Item Count: 41935
0/5988, Runtime: 20.636539220809937sec
599/5988, Runtime: 1632.6383023262024sec
1198/5988, Runtime: 36927.996894836426sec
1797/5988, Runtime: 3193.4879751205444sec
2396/5988, Runtime: 2394.6998221874237sec
2994/5988, Runtime: 5126.698722362518sec
3593/5988, Runtime: 1496.7094008922577sec
4192/5988, Runtime: 3766.148491859436sec
4791/5988, Runtime: 2745.7958569526672sec
5390/5988, Runtime: 1860.202401638031sec
Basket Shape: (5988, 41935)
Item Count: 5294
0/5988, Runtime: 0.9305014610290527sec
599/5988, Runtime: 498.0235173702

In [6]:
%%time
# The scores
scores_df = movies_df[['avg_vote', 'metascore', 'user_reviews', 'critic_reviews', 'votes']]
scores_df = scores_df.dropna(subset=['avg_vote', 'metascore', 'user_reviews', 'critic_reviews'])
scores_df = scores_df[scores_df['votes'] >= 10000]
scores_df = scores_df.drop(columns=['votes'])

title_avgscore = make_marketbasket_from_interval(scores_df, 'avg_vote', 0.0, 10.0, 2.5)
title_metascore = make_marketbasket_from_interval(scores_df, 'metascore', 0.0, 10.0, 2.5)
title_userscore = make_marketbasket_from_interval(scores_df, 'user_reviews', 0.0, 10.0, 2.5)
title_criticscore = make_marketbasket_from_interval(scores_df, 'critic_reviews', 0.0, 10.0, 2.5)

Discretized Basket: ['avg_vote < 0.0', 'avg_vote in [0.0, 2.5)', 'avg_vote in [2.5, 5.0)', 'avg_vote in [5.0, 7.5)', 'avg_vote in [7.5, 10.0)', 'avg_vote >= 10.0']
0/5988, Runtime: 0.0059413909912109375sec
599/5988, Runtime: 1.553161382675171sec
1198/5988, Runtime: 1.445366382598877sec
1797/5988, Runtime: 1.5942888259887695sec
2396/5988, Runtime: 1.285433292388916sec
2994/5988, Runtime: 1.3903319835662842sec
3593/5988, Runtime: 1.256376028060913sec
4192/5988, Runtime: 1.222736120223999sec
4791/5988, Runtime: 1.2293050289154053sec
5390/5988, Runtime: 1.283907175064087sec
Transaction Shape: (5988, 6)
Discretized Basket: ['metascore < 0.0', 'metascore in [0.0, 2.5)', 'metascore in [2.5, 5.0)', 'metascore in [5.0, 7.5)', 'metascore in [7.5, 10.0)', 'metascore >= 10.0']
0/5988, Runtime: 0.003997325897216797sec
599/5988, Runtime: 1.2419464588165283sec
1198/5988, Runtime: 1.2629659175872803sec
1797/5988, Runtime: 1.2909059524536133sec
2396/5988, Runtime: 1.2806110382080078sec
2994/5988, Runti

In [7]:
title_avgscore.describe()

,avg_vote < 0.0,"avg_vote in [0.0, 2.5)","avg_vote in [2.5, 5.0)","avg_vote in [5.0, 7.5)","avg_vote in [7.5, 10.0)",avg_vote >= 10.0
count,5988,5988,5988,5988,5988,5988
unique,1,2,2,2,2,1
top,False,False,False,True,False,False
freq,5988,5976,5701,4699,4998,5988


### Organize the Data Into different categories

##### Director Stats; why we can't use it alone